# Part 4: VGG on cifar10

In [13]:
import numpy as np
import pandas as pd

from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D 
from tensorflow.python.keras.preprocessing.image import array_to_img
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint

import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy

from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

In [14]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [15]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test)=datasets.cifar10.load_data()

We can check the shape of new data and see that our images are 32×32 pixels, so we need to add a new axis, which will represent a number of channels. Also, it is important to do one-hot encoding of labels and normalization of input images.

In [16]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_train[0].shape, 'image shape')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
(32, 32, 3) image shape


In [17]:
# Convert class vectors to binary class matrices.

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [18]:
# Data normalization
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [19]:
#define VGG
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 8, 8, 128)        

In [20]:
adam = Adam(lr=1e-4, decay=1e-6)
#model.compile(optimizer=adam, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.compile(adam, 'categorical_crossentropy', metrics=['accuracy'])

After creating a model, we need to train its parameters to make it powerful. Let’s train the model for a given number of epochs.


In [21]:
# Place the logs in a timestamped subdirectory
# This allows to easy select different training runs
# In order not to overwrite some data, it is useful to have a name with a timestamp
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Specify the callback object
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# tf.keras.callback.TensorBoard ensures that logs are created and stored
# We need to pass callback object to the fit method
# The way to do this is by passing the list of callback objects, which is in our case just one

In [22]:
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test),callbacks=[tensorboard_callback])

Epoch 1/20
  1/782 [..............................] - ETA: 0s - loss: 2.3030 - accuracy: 0.0781WARNING:tensorflow:From /Users/zhangmaoyu/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
782/782 [==============================] - 139s 178ms/step - loss: 1.7554 - accuracy: 0.3514 - val_loss: 1.4745 - val_accuracy: 0.4564
Epoch 2/20
782/782 [==============================] - 137s 175ms/step - loss: 1.3810 - accuracy: 0.4998 - val_loss: 1.2891 - val_accuracy: 0.5390
Epoch 3/20
782/782 [==============================] - 139s 178ms/step - loss: 1.2142 - accuracy: 0.5636 - val_loss: 1.1581 - val_accuracy: 0.5789
Epoch 4/20
782/782 [==============================] - 139s 178ms/step - loss: 1.0925 - accuracy: 0.6098 - val_loss: 1.0953 - val_accuracy: 0.6118
Epoch 5/20
782/782 [=============

In [23]:
%tensorboard --logdir=logs/fit

Reusing TensorBoard on port 6008 (pid 30593), started 2:37:14 ago. (Use '!kill 30593' to kill it.)

From the above results, it can be found that VGG performs better than Lenet on cifar10 data. Compared with Lenet, I think VGG will train better,since the training accuracy is up to 96% after 20 epoches,and testing accuracy is about to 70%,this is quite larger than that in Lenet whose testing accuraacy is only about to 50%. And also I think the accuracy will be higher with the increase of the epoches,so that I hold the view that VGG performs better than Lenet on the cifar10 data set.

Next,we Train the model again with smaller learning rate to see the change of the loss and accuracy.

## Training the model again with smaller learning rate

In [24]:
adam = Adam(lr=1e-5, decay=1e-6)
model.compile(adam, 'categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test),callbacks=[tensorboard_callback])

Epoch 1/20
782/782 [==============================] - 144s 185ms/step - loss: 0.0292 - accuracy: 0.9947 - val_loss: 1.3202 - val_accuracy: 0.7463
Epoch 2/20
782/782 [==============================] - 145s 186ms/step - loss: 0.0126 - accuracy: 0.9988 - val_loss: 1.4140 - val_accuracy: 0.7493
Epoch 3/20
782/782 [==============================] - 139s 177ms/step - loss: 0.0080 - accuracy: 0.9995 - val_loss: 1.4909 - val_accuracy: 0.7515
Epoch 4/20
782/782 [==============================] - 139s 178ms/step - loss: 0.0053 - accuracy: 0.9997 - val_loss: 1.5836 - val_accuracy: 0.7489
Epoch 5/20
782/782 [==============================] - 139s 178ms/step - loss: 0.0035 - accuracy: 0.9999 - val_loss: 1.6723 - val_accuracy: 0.7478
Epoch 6/20
782/782 [==============================] - 141s 181ms/step - loss: 0.0026 - accuracy: 0.9999 - val_loss: 1.7237 - val_accuracy: 0.7476
Epoch 7/20
782/782 [==============================] - 139s 177ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 1.7949 -

In [26]:
%tensorboard --logdir=logs/fit

Reusing TensorBoard on port 6008 (pid 30593), started 3:26:57 ago. (Use '!kill 30593' to kill it.)

When we train the model again with smaller learning rate,We can achieve optimal accuracy much faster，and the training accuracy can almost up to 100%,and the testing accuracy is larger than the  1e-4 learning rate in the fisrt part,the training loss is become smaller, but the validation loss is much larger than  the 1e-4 learning rate in the fisrt part.At some point, I think there might be an overfitting situation. Therefore,we can conclude that the VGG with perfect hyper-parameter (which is a mannual process )can perform much better than Lenet on cifar10 data，However, we also need to pay attention to the possibility that the model may overfit.

# conclusion

In general, both VGG and Lenet performed better in MINST data than in CIFAR10 data set. VGG neural network has more layers and is more complex, so the training effect will be better than Lenet. However, there may be overfitting problem, so we need to select appropriate hyperparameters for the model.